# Data Load

In [ ]:
import datetime as dt  # Python standard library datetime  module
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from tqdm import tqdm_notebook
import pickle
import time
from tqdm import tqdm
from sklearn import metrics
import gc
import statsmodels.formula.api as smf
import statsmodels.api as sm
from collections import Counter#<---value count for list
from sklearn.model_selection import StratifiedKFold

In [ ]:
#Select the target species
file_id="nutwoo"
bird_name="Nuttall's Woodpecker"
bcr_id='32'

file_id="recwoo"
bird_name="Red-cockaded Woodpecker"
bcr_id='27'

file_id="lewwoo"
bird_name="Lewis’s Woodpecker"
bcr_id='9 and 10'

In [ ]:
PATH='/content/drive/My Drive/Colab Notebooks/dissertation/'
ebird_ss=pd.read_csv(PATH+'ebird_ss_'+file_id+'_add30yMonth.csv')

## Define useful functions

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        if col != 'time':
            col_type = df[col].dtypes
            if col_type in numerics:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
import itertools
def plot_confusion_matrix(cm,
                          classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix very prettily.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    fig, ax = plt.subplots(figsize=(3, 3))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)

    # Specify the tick marks and axis text
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    # The data formatting
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    
    # Print the text of the matrix, adjusting text colour for display
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
from sklearn.metrics import roc_curve, auc
def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value
    (Source: https://stackoverflow.com/questions/56203889/how-to-get-the-optimal-threshold-from-roc-curve-in-python)   
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 

## Drop/fix NaN values

In [ ]:
#Check
print((ebird_ss.isna().describe().loc['unique']==2).sort_values())

In [ ]:
if file_id=="nutwoo":
    # For Nuttall's Woodpecker
    print(ebird_ss.prec30_cv.isna().value_counts())
    print(ebird_ss.prec180_cv.isna().value_counts())
    print(ebird_ss.observation_count.isna().value_counts())
    ebird_ss.loc[ebird_ss['prec30_cv'].isna(),'prec30_cv']=0
    ebird_ss.loc[ebird_ss['prec180_cv'].isna(),'prec180_cv']=0
    ebird_ss.drop(columns=['observation_count'],inplace=True)

elif file_id=="recwoo":
    # For "Red-cockaded Woodpecker"
    print(ebird_ss.prec30_cv.isna().value_counts())
    print(ebird_ss.elevation_median.isna().value_counts())
    print(ebird_ss.elevation_sd.isna().value_counts())
    print(ebird_ss.observation_count.isna().value_counts())
    ebird_ss.drop(columns=['observation_count'],inplace=True)
    ebird_ss.dropna(inplace=True)
    ebird_ss.reset_index(drop=True,inplace=True)

elif file_id=="lewwoo":
    # For "Lewis’s Woodpecker"
    print(ebird_ss.prec30_cv.isna().value_counts())
    print(ebird_ss.observation_count.isna().value_counts())
    ebird_ss.loc[ebird_ss['prec30_cv'].isna(),'prec30_cv']=0
    ebird_ss.drop(columns=['observation_count'],inplace=True)

else:
    print('Missing file_id')

In [ ]:
ebird_ss=reduce_mem_usage(ebird_ss)

## Set variables for use

In [ ]:
variables_base=['species_observed',
'time_observations_started',
'duration_minutes',
'effort_distance_km',
 'number_observers',
 'bio1',
 'bio2',
 'bio3',
 'bio4',
 'bio5',
 'bio6',
 'bio7',
 'bio8',
 'bio9',
 'bio10',
 'bio11',
 'bio12',
 'bio13',
 'bio14',
 'bio15',
 'bio16',
 'bio17',
 'bio18',
 'bio19',
 'prec30_mean',
 'prec180_mean',
 'prec365_mean',
 'prec730_mean',
 'prec1095_mean',
 'prec1460_mean',
 'prec1825_mean',
 'tmp30_mean',
 'tmp30_std',
 'tmp180_mean',
 'tmp180_std',
 'tmp365_mean',
 'tmp365_std',
 'tmp730_mean',
 'tmp730_std',
 'tmp1095_mean',
 'tmp1095_std',
 'tmp1460_mean',
 'tmp1460_std',
 'tmp1825_mean',
 'tmp1825_std',
 'tmax30_mean',
 'tmax30_std',
 'tmax180_mean',
 'tmax180_std',
 'tmax365_mean',
 'tmax365_std',
 'tmax730_mean',
 'tmax730_std',
 'tmax1095_mean',
 'tmax1095_std',
 'tmax1460_mean',
 'tmax1460_std',
 'tmax1825_mean',
 'tmax1825_std',
 'tmin30_mean',
 'tmin30_std',
 'tmin180_mean',
 'tmin180_std',
 'tmin365_mean',
 'tmin365_std',
 'tmin730_mean',
 'tmin730_std',
 'tmin1095_mean',
 'tmin1095_std',
 'tmin1460_mean',
 'tmin1460_std',
 'tmin1825_mean',
 'tmin1825_std',
 'prec30_cv',
 'prec180_cv',
 'prec365_cv',
 'prec730_cv',
 'prec1095_cv',
 'prec1460_cv',
 'prec1825_cv',
 'tmin_30y_monthly',
 'tmax_30y_monthly',
 'tavg_30y_monthly',
 'prec_30y_monthly',
 'srad_30y_monthly',
 'wind_30y_monthly',
 'vapr_30y_monthly',
 'pland_00_water',
 'pland_01_evergreen_needleleaf',
 'pland_02_evergreen_broadleaf',
 'pland_03_deciduous_needleleaf',#<--remove for nutwoo
 'pland_04_deciduous_broadleaf',
 'pland_05_mixed_forest',
 'pland_06_closed_shrubland',
 'pland_07_open_shrubland',
 'pland_08_woody_savanna',
 'pland_09_savanna',
 'pland_10_grassland',
 'pland_11_wetland',
 'pland_12_cropland',
 'pland_13_urban',
 'pland_14_mosiac',
 'pland_15_barren',
'elevation_median',
'elevation_sd'
]
if file_id=="nutwoo":
    variables_base.remove('pland_03_deciduous_needleleaf')

# MLP

## Data normalization

In [ ]:
from sklearn import preprocessing
variables=variables_base.copy()
x_base = ebird_ss[variables].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(x_base)
ebird_ss_nn = min_max_scaler.transform(x_base)
ebird_ss_nn = pd.DataFrame(ebird_ss_nn,columns=variables)

## Define the model

In [ ]:
import torch.nn as nn
class CustomLinear(nn.Module):
    def __init__(self, in_channels, bias=True, p=0.1):
        super().__init__()
        self.linear = nn.Linear(in_channels, 512, bias)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p)
        self.bn1 = nn.BatchNorm1d(num_features=512)
        self.fc1 = nn.Linear(512,1)
        
    def forward(self, x):
        x = self.linear(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.drop(x)
        x = self.fc1(x)
        return x

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

## Train the model

In [ ]:
import copy
import torch
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=72).split(ebird_ss_nn, ebird_ss_nn[['species_observed']]))

batch_size = 512
train_epochs = 100

AUC_ls = []
result_ls = []
model_tuning_ls_test = []
test_preds_ls = []

for i, (train_idx, test_idx) in enumerate(splits):
    model_tuning_ls_valid = []
    print(f'+++++++Start for {i+1} fold test data+++++++')
    variables=variables_base.copy()

    train_x = ebird_ss_nn.iloc[train_idx,:]
    test_x = ebird_ss_nn.iloc[test_idx,:]

    train_x=train_x[variables]
    test_x=test_x[variables]
    train_X=train_x[variables[1:]]
    train_y=train_x['species_observed']
    test_X=test_x[variables[1:]]
    test_y=test_x['species_observed']


    train_preds = np.zeros((len(train_X)))
    test_preds=[]

    x_test_cuda = torch.tensor(test_X.values, dtype=torch.float32).cuda()
    test = torch.utils.data.TensorDataset(x_test_cuda)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

    splits_val = list(StratifiedKFold(n_splits=4, shuffle=True, random_state=72).split(train_X, train_y))
    for j, (train2_idx, val_idx) in enumerate(splits_val):
        print(f'=======Start for {j+1} fold valid data in {i+1} fold test data=====')

        train2_x = train_x.iloc[train2_idx,:]
        valid_x = train_x.iloc[val_idx,:]

        train2_X=train2_x[variables[1:]]
        train2_y=train2_x['species_observed']
        valid_X=valid_x[variables[1:]]
        valid_y=valid_x['species_observed']

        x_train_fold = torch.tensor(train2_X.to_numpy(), dtype=torch.float32).cuda()
        y_train_fold = torch.tensor(train2_y[:, np.newaxis], dtype=torch.float32).cuda()

        x_val_fold = torch.tensor(valid_X.to_numpy(), dtype=torch.float32).cuda()
        y_val_fold = torch.tensor(valid_y[:, np.newaxis], dtype=torch.float32).cuda()
        
        if file_id=="nutwoo":
            net=CustomLinear(103)
        else:
            net=CustomLinear(104)
        model = net
        model.cuda()
        
        loss_fn = torch.nn.BCEWithLogitsLoss(reduction="sum")
        optimizer = torch.optim.Adam(model.parameters(),lr=0.001,)
        
        train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
        valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)
        
        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
        valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
        
        print(f'Fold {i + 1}-Validation {j + 1}')
        
        ave_val_min=999.
        count=0
        for epoch in range(train_epochs):
            start_time = time.time()
            
            model.train()
            avg_loss = 0.

            for x_batch, y_batch in tqdm(train_loader, disable=True):
                 y_pred = model(x_batch)
                loss = loss_fn(y_pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                avg_loss += loss.item() / len(train_loader)
            
            model.eval()
            valid_preds_fold = np.zeros((x_val_fold.size(0)))
            test_preds_fold = np.zeros(len(test_x))
            avg_val_loss = 0.

            for k, (x_batch, y_batch) in enumerate(valid_loader):
                y_pred = model(x_batch).detach()
                avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
                valid_preds_fold[k * batch_size:(k+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]
            
            elapsed_time = time.time() - start_time 
            print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s'.format(
                epoch + 1, train_epochs, avg_loss, avg_val_loss, elapsed_time))
            
            if ave_val_min >= avg_val_loss:
                ave_val_min = avg_val_loss
                best_model = copy.deepcopy(model)
                best_epoch = epoch + 1
                count=0
            else:
                count+=1
            if count==10:
                print('Not updated during the last 10 epochs')
                break

        for l, (x_batch,) in enumerate(test_loader):
            y_pred = best_model(x_batch).detach()
            test_preds_fold[l * batch_size:(l+1) * batch_size] += sigmoid(y_pred.cpu().numpy())[:, 0]
        print(f'!!!Best model is at Epoch {best_epoch}!!!')

        test_preds.append(test_preds_fold)
    test_preds_ls.append(test_preds)

with open('test_preds_ls_'+file_id+'.pkl', 'wb') as handle:
    pickle.dump(test_preds_ls, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Show ROC

In [ ]:
from sklearn.model_selection import StratifiedKFold
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=72).split(ebird_ss, ebird_ss[['species_observed']]))

fig,ax=plt.subplots(1,figsize=(5,4))

AUC_ls=[]
for i, ((train_idx, test_idx),test_preds) in enumerate(zip(splits,test_preds_ls)):
    variables=variables_base.copy()
    
    train_x = ebird_ss.iloc[train_idx,:]
    test_x = ebird_ss.iloc[test_idx,:]

    train_x=train_x[variables]
    test_x=test_x[variables]

    train_X=train_x[variables[1:]]
    train_y=train_x['species_observed']
    test_X=test_x[variables[1:]]
    test_y=test_x['species_observed']

    pred = np.mean(test_preds,axis=0)
    tmp1=pd.DataFrame(pred,columns=["prediction"])
    tmp2=pd.DataFrame(test_y.values.reshape(-1,1),columns=['Actual']).astype(int)
    df_RF=pd.concat([tmp1,tmp2],axis=1)
    
    rf_obs=test_x.sort_values('species_observed').astype(float)['species_observed'].values
    rf_pred=df_RF['prediction'].values


    fpr, tpr, thresholds = metrics.roc_curve(rf_obs,rf_pred, pos_label=None)
    AUC=metrics.auc(fpr, tpr)
    AUC_ls.append(AUC)
    print(f'AUC of valid data:{AUC:0.3f}')

    plt.plot([0,1],[0,1],'k--')
    auc_val=str(round(metrics.auc(fpr, tpr),3)).ljust(5, '0')
    plt.plot(fpr,tpr,label=f'[AUC={auc_val}] {i+1}-fold')
    plt.xlabel('False positive rate (1-specificity)')
    plt.ylabel('True positive rate (sensitivity)')
    plt.title('ROC curve')

print(f'Mean: {np.mean(AUC_ls):0.4f}')
print(f'SD: {np.std(AUC_ls):0.4f}')
plt.legend(loc='best')
fig.savefig(f'mlp_AUC_{file_id}.png',bbox_inches='tight')
plt.show()

## Sensitivity, specificity, threshold, confusion matrix

In [ ]:
from sklearn.model_selection import StratifiedKFold
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=72).split(ebird_ss, ebird_ss[['species_observed']]))

sensitivity_ls=[]
specificity_ls=[]
threshold_ls=[]

for i, ((train_idx, test_idx),test_preds) in enumerate(zip(splits,test_preds_ls)):   
    variables=variables_base.copy()
    
    train_x = ebird_ss.iloc[train_idx,:]
    test_x = ebird_ss.iloc[test_idx,:]

    train_x=train_x[variables]
    test_x=test_x[variables]

    train_X=train_x[variables[1:]]
    train_y=train_x['species_observed']
    test_X=test_x[variables[1:]]
    test_y=test_x['species_observed']

    pred = np.mean(test_preds,axis=0)
    tmp1=pd.DataFrame(pred,columns=["prediction"])
    tmp2=pd.DataFrame(test_y.values.reshape(-1,1),columns=['Actual']).astype(int)
    df_RF=pd.concat([tmp1,tmp2],axis=1)
    
    rf_obs=test_x.sort_values('species_observed').astype(float)['species_observed'].values
    rf_pred=df_RF['prediction'].values
    
    threshold = Find_Optimal_Cutoff(rf_obs,rf_pred)
    print(f'threshold: {np.round(threshold[0],3)}')
    threshold_ls.append(threshold)
    rf_pred_disc=rf_pred>threshold[0]

    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(rf_obs,rf_pred_disc,labels=[1,0])

    sensitivity1 = cm[0,0]/(cm[0,0]+cm[0,1])
    print('Sensitivity : ', np.round(sensitivity1,3) )
    sensitivity_ls.append(sensitivity1)

    specificity1 = cm[1,1]/(cm[1,0]+cm[1,1])
    print('Specificity : ', np.round(specificity1,3))
    specificity_ls.append(specificity1)
    print(f'{np.round(sensitivity1,3)}&{np.round(specificity1,3)}&{np.round(threshold[0],3)}')

    classes=['True','False']
    plot_confusion_matrix(cm,classes,normalize=False)
    
    if i==0:
        plt.savefig(f'mlp_cm_{file_id}_{i+1}_fold.png',bbox_inches='tight')
    plt.show()

print(f'Threshold: {np.mean(threshold_ls):0.4f}')
print(f'Sensitivity: {np.mean(sensitivity_ls):0.4f}')
print(f'Specificity: {np.mean(specificity_ls):0.4f}')


print(f'Threshold: {np.std(threshold_ls):0.4f}')
print(f'Sensitivity: {np.std(sensitivity_ls):0.4f}')
print(f'Specificity: {np.std(specificity_ls):0.4f}')

## Calibration plots

In [ ]:
from sklearn.model_selection import StratifiedKFold
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=72).split(ebird_ss, ebird_ss[['species_observed']]))

fig,axes=plt.subplots(1,3,figsize=(15,5))
for i, ((train_idx, test_idx),test_preds) in enumerate(zip(splits,test_preds_ls)):
    variables=variables_base.copy()
    
    train_x = ebird_ss.iloc[train_idx,:]
    test_x = ebird_ss.iloc[test_idx,:]

    train_x=train_x[variables]
    test_x=test_x[variables]

    train_X=train_x[variables[1:]]
    train_y=train_x['species_observed']
    test_X=test_x[variables[1:]]
    test_y=test_x['species_observed']

    pred = np.mean(test_preds,axis=0)
    tmp1=pd.DataFrame(pred,columns=["prediction"])
    tmp2=pd.DataFrame(test_y.values.reshape(-1,1),columns=['Actual']).astype(int)
    df_RF=pd.concat([tmp1,tmp2],axis=1)
    
    rf_obs=test_x.sort_values('species_observed').astype(float)['species_observed'].values
    rf_pred=df_RF['prediction'].values

    bin=0.05
    df_calib=pd.DataFrame([pd.cut(rf_pred, bins=np.arange(0,1+bin,bin),include_lowest=False),rf_obs],
                        columns=('prediction','observed'))

    scatter=axes[0].scatter(np.arange(bin/2,1.00+bin/2,bin),
            df_calib.groupby('prediction').mean()['observed'].values,
                alpha=0.5,label=f'{i+1}-fold')
    axes[0].set_xlabel('Predicted encounter rate')
    axes[0].set_ylabel('Observed encounter rate')
    axes[0].set_title(f'Calibration plot (group size = {bin})')
    axes[0].plot([0,1],[0,1],'k--')

    bin=0.02
    df_calib=pd.DataFrame([pd.cut(rf_pred, bins=np.arange(0,1+bin,bin),include_lowest=False),rf_obs],
                        columns=('prediction','observed'))

    scatter=axes[1].scatter(np.arange(bin/2,1.00+bin/2,bin),
            df_calib.groupby('prediction').mean()['observed'].values,
                alpha=0.5,label=f'{i+1}-fold')
    axes[1].set_xlabel('Predicted encounter rate')
    axes[1].set_ylabel('Observed encounter rate')
    axes[1].set_title(f'Calibration plot (group size = {bin})')
    axes[1].plot([0,1],[0,1],'k--')

    axes[2].plot(np.arange(bin/2,1.00+bin/2,bin),
                df_calib['prediction'].value_counts(normalize=True).sort_index().values,alpha=0.5,marker="*",label=f'{i+1}-fold')
    axes[2].set_xlabel('Predicted encounter rate')
    axes[2].set_ylabel('Frequency (percentage)')
    axes[2].set_title(f'Distribution of the prediction (group size = {bin})')

axes[0].legend(loc='best')
axes[1].legend(loc='best')
axes[2].legend(loc='best')
fig.savefig(f'mlp_calib_all_{file_id}.png',bbox_inches='tight')
plt.show()